# Natural Posterior Network (NatPN) Tutorial in PyTorch

## A Detailed, Notebook-Ready Walkthrough with OOD Uncertainty Check

This notebook is a step-by-step implementation and explanation of a simple Natural Posterior Network (NatPN) for classification with PyTorch.

NatPN is a model that:

- Predicts a distribution over class probabilities, not just point predictions.
- Uses a conjugate prior (Dirichlet) for the categorical likelihood.
- Uses a normalizing flow over a latent space to turn density into evidence.
- Produces higher uncertainty for inputs that are far from the training data, without needing OOD samples in training.

We will build this for a simple setting:

- **In-distribution (ID) dataset:** MNIST (handwritten digits).
- **Out-of-distribution (OOD) dataset:** FashionMNIST (clothing images).

We will:

- Explain and implement the NatPN components (encoder, flow-based density, Dirichlet posterior).
- Define and explain a Bayesian NatPN loss.
- Implement a unified training function that could live in a `torch.py` module.
- Train the model, evaluate accuracy, and check uncertainty for ID vs OOD.

---

## Table of Contents

1. **Imports & Setup**
2. **Data Preparation**
3. **Model Definition (NatPN Components)**
   - Encoder: \( x \rightarrow z \)
   - Radial Flow Layer
   - Radial Flow Density
   - NatPNClassifier: Combining Everything
4. **NatPN Loss, Uncertainty & Unified Training Function**
   - NatPN Loss for Classification
   - Predictions & Uncertainty from Dirichlet
   - Unified NatPN Training Function
5. **Training, Evaluation & OOD Uncertainty Check**
   - Initialize Model & Train
   - Accuracy on MNIST (ID)
   - OOD Check: MNIST vs FashionMNIST
   - Inspect Individual Batches
6. **Summary**

# 1. Imports & Setup

## Explanation

In this section we:

### Import core PyTorch libraries:

- `torch` and `torch.nn` for tensors and neural network layers.
- `DataLoader` to iterate over datasets in mini-batches.

### Import torchvision:

- `torchvision.datasets` provides pre-defined datasets like MNIST and FashionMNIST.
- `torchvision.transforms` provides transformations such as converting images to tensors.

### Import `Dirichlet` from `torch.distributions`:

- We use this for working with Dirichlet distributions, which serve as the conjugate prior/posterior for categorical likelihoods.

### Decide on a device:

- `"cuda"` if a GPU is available.
- `"cpu"` otherwise.

### Optionally import typing helpers:

- `Optional`, `Tuple`, etc., for cleaner and more descriptive type hints.


In [ ]:
import math

import torch
from torch import nn
from torch.distributions import Dirichlet
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as T

# Choose device: use GPU if available, otherwise CPU.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# 2. Data Preparation

## Explanation

NatPN is trained like a standard classifier, but with uncertainty-aware outputs.

We need **3 datasets**:

### 1. Training data (ID)

- **MNIST**, `train=True`
- Contains handwritten digits (0–9)

### 2. Test data (ID)

- **MNIST**, `train=False`
- Used for evaluating accuracy and in-distribution uncertainty

### 3. OOD data

- **FashionMNIST**, `train=False`
- Has the same image shape (**28×28, grayscale**) but represents clothing instead of digits  
- This makes it suitable for out-of-distribution uncertainty evaluation

---

## What we do

### Apply a transform:

- Use `ToTensor()` to convert PIL images into PyTorch tensors  
- Produces tensors of shape **[1, 28, 28]**  
- Pixel values are scaled to **[0, 1]**

---

## Wrap datasets in `DataLoader` objects:

- `batch_size = 256` to process 256 images per iteration
- `shuffle = True` for training so data order is randomized
- `shuffle = False` for evaluation since order does not matter


In [ ]:
# Transform with correct normalization for MNIST & FashionMNIST
transform = T.Compose(
    [
        T.ToTensor(),
        T.Normalize((0.1307,), (0.3081,)),  # IMPORTANT: consistent normalization
    ],
)

# In-distribution (ID) dataset: MNIST (train split)
train_data = torchvision.datasets.MNIST(
    root="~/datasets",
    train=True,
    download=True,
    transform=transform,
)

# In-distribution (ID) dataset: MNIST (test split)
test_data = torchvision.datasets.MNIST(
    root="~/datasets",
    train=False,
    download=True,
    transform=transform,
)

# Out-of-distribution (OOD) dataset: FashionMNIST
ood_data = torchvision.datasets.FashionMNIST(
    root="~/datasets",
    train=False,
    download=True,
    transform=transform,
)

# DataLoaders
train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)
ood_loader = DataLoader(ood_data, batch_size=256, shuffle=False)

print("Loaded MNIST (ID) and FashionMNIST (OOD).")
print(f"Train samples: {len(train_data)}, Test samples: {len(test_data)}, OOD samples: {len(ood_data)}")

# 3. Model Definition (NatPN Components)

NatPN consists of three main pieces:

---

## **1. Encoder  fφ**

The encoder maps an input x into a latent representation z:

fφ(x) → z

---

## **2. Density Model  p(z)**

A normalizing-flow–based density estimator that produces a log-density value:

log p(z)

This density quantifies how likely the latent representation z is under the learned distribution and acts as evidence.

---

## **3. Dirichlet Head (Decoder)**

The decoder uses the latent representation z together with density-based evidence to generate the Dirichlet parameters:

α(x)

These parameters define a Dirichlet distribution over class probabilities.

---

## **With the Dirichlet parameters α(x), we can:**

### **Compute posterior mean class probabilities**
Used as calibrated class probabilities.

### **Compute uncertainty measures**
Such as:

- Entropy  
- Total evidence  
- Dirichlet concentration  

These quantify predictive uncertainty and help distinguish between in-distribution and out-of-distribution samples.


# 3.1 Encoder:  x → z

## Explanation

The encoder is a simple fully connected neural network.

---

### Input

Images have shape **[B, 1, 28, 28]**, where **B** is the batch size.

We flatten each image into **[B, 784]** so it can pass through linear layers.

---

### Network structure

- Linear(784 → 256) + ReLU  
- Linear(256 → latent_dim)

---

### Latent representation

The output z lives in a low-dimensional latent space.

We choose **latent_dim = 2** to keep the normalizing flow simple and (optionally) easy to visualize.

---

### Purpose

The encoder learns a mapping from images into a latent space where the normalizing flow can model a meaningful density.


In [ ]:
class Encoder2(nn.Module):
    """Simple encoder mapping MNIST images to a low-dimensional latent vector z."""

    def __init__(self, latent_dim: int = 2) -> None:  # noqa: D107
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),  # [B, 1, 28, 28] -> [B, 784]
            nn.Linear(28 * 28, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim),  # [B, latent_dim]
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Encode a batch of images into latent vectors z.

        Args:
            x: Tensor of shape [B, 1, 28, 28].

        Returns:
            z: Tensor of shape [B, latent_dim].
        """
        return self.net(x)

# 3.2 Radial Flow Layer

## Explanation

A normalizing flow is a sequence of invertible transformations that map a simple base distribution (such as a standard normal) into a more complex one.

A **radial flow** is one such transformation, defined as:

f(z) = z + β · h(r) · (z − z₀)

where:

- z₀ is a learnable center  
- r = ‖z − z₀‖ is the Euclidean distance from the center  
- h(r) = 1 / (α + r)  
- α > 0 and β > −α are learnable scalars whose constraints ensure invertibility

---

## Key properties

- The transformation warps space **radially** around the center z₀.  
- The **log-determinant of the Jacobian** can be computed analytically, which is essential for density estimation in normalizing flows.

---

## RadialFlowLayer behavior

Input:

- z of shape **[B, D]**

Outputs:

- **z_new**: the transformed latent vectors  
- **log_abs_det**: the log-determinant of the Jacobian for each sample  

---

## Parameter constraints using softplus

To ensure the flow remains invertible:

- α = softplus(α′)  → ensures α > 0  
- β = −α + softplus(β′) → ensures β > −α


In [ ]:
class RadialFlowLayer(nn.Module):
    """Single radial flow layer for a latent vector z ∈ R^D."""

    def __init__(self, dim: int) -> None:  # noqa: D107
        super().__init__()
        self.dim = dim

        # Learnable parameters:
        # - x0: center of the radial transformation (vector in R^D)
        # - alpha_prime, beta_prime: unconstrained scalars that we transform to valid alpha, beta
        self.x0 = nn.Parameter(torch.zeros(dim))
        self.alpha_prime = nn.Parameter(torch.zeros(1))
        self.beta_prime = nn.Parameter(torch.zeros(1))

    def forward(self, z: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        """Apply the radial flow to latent inputs z.

        Args:
            z: Tensor of shape [B, D].

        Returns:
            z_new: Transformed latent tensor, shape [B, D].
            log_abs_det: Log-absolute determinant of the Jacobian, shape [B].
        """
        # Ensure alpha > 0 and beta > -alpha for invertibility
        alpha = torch.nn.functional.softplus(self.alpha_prime)  # scalar > 0
        beta = -alpha + torch.nn.functional.softplus(self.beta_prime)  # scalar > -alpha

        # z0 is the learnable center (broadcast to [B, D])
        x0 = self.x0  # [D]

        # Difference from the center
        diff = z - x0  # [B, D]
        r = diff.norm(dim=-1)  # Distance to center, shape [B]

        # Radial flow scalar functions h(r) and h'(r)
        h = 1.0 / (alpha + r)  # [B]
        h_prime = -h * h  # [B]
        beta_h = beta * h  # [B]

        # Apply the radial flow transformation:
        z_new = z + beta_h.unsqueeze(-1) * diff  # [B, D]

        # Log determinant of the Jacobian:
        # formula derived in Rezende & Mohamed (2015)
        term1 = (self.dim - 1) * torch.log1p(beta_h)  # [B]
        term2 = torch.log1p(beta_h + beta * h_prime * r)  # [B]
        log_abs_det = term1 + term2  # [B]

        return z_new, log_abs_det

# 3.3 Radial Flow Density

## Explanation

We now build the full flow-based density **p(z)** by stacking multiple radial flow layers.

---

## Base distribution

We start from a simple base distribution:

z₀ ∼ 𝒩(0, I)

This is a standard multivariate normal with zero mean and identity covariance.

---

## Applying L radial flow layers

Each flow layer transforms its input:

zₖ₊₁ = fₖ(zₖ), k = 0, 1, …, L−1

After applying all L layers, we obtain zᴸ — the final transformed latent vector.

---

## Change of variables formula

The log-density of the final output zᴸ is computed using:

log p(zᴸ) = log p₀(z₀) + Σₖ₌₀⁽ᴸ⁻¹⁾ log |det ∂fₖ/∂zₖ|

This combines:

- The log-density of the base distribution p₀(z₀)  
- The sum of log-determinants of each flow’s Jacobian

---

## Behavior in code

### **forward(x)**  
Applies all radial flows sequentially and returns the transformed z.

### **log_prob(x)**  
- Computes the transformed z  
- Sums all log-Jacobian determinants  
- Computes the base log-probability under a standard Normal  
- Returns the total:

log p(x)


In [ ]:
class RadialFlowDensity(nn.Module):
    """Normalizing flow density p(z) using a stack of radial flows."""

    def __init__(self, dim: int, flow_length: int = 4) -> None:  # noqa: D107
        super().__init__()
        self.dim = dim
        self.layers = nn.ModuleList([RadialFlowLayer(dim) for _ in range(flow_length)])

        # Constant term for log N(z|0, I): -0.5 * D * log(2π)
        self.log_base_const = -0.5 * self.dim * math.log(2 * math.pi)

    def forward(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        """Apply all flow layers to x.

        Args:
            x: Tensor of shape [B, D].

        Returns:
            z: Transformed latent tensor after all flows, shape [B, D].
            sum_log_jac: Summed log-det Jacobian across flows, shape [B].
        """
        z = x
        sum_log_jac = torch.zeros(z.size(0), device=z.device)

        for layer in self.layers:
            z, log_j = layer(z)
            sum_log_jac = sum_log_jac + log_j

        return z, sum_log_jac

    def log_prob(self, x: torch.Tensor) -> torch.Tensor:
        """Compute log p(x) under the flow-based density.

        Args:
            x: Tensor of shape [B, D].

        Returns:
            logp: Log-density log p(x), shape [B].
        """
        # Apply flow
        z, sum_log_jac = self.forward(x)

        # Base log-prob under N(0, I): -0.5 * (D * log(2π) + ||z||^2)
        base_logp = self.log_base_const - 0.5 * (z**2).sum(dim=-1)  # [B]

        # Add the log-determinant of the Jacobian
        logp = base_logp + sum_log_jac  # [B]
        return logp

# 3.4 NatPNClassifier: Combining Everything

## Explanation

The **NatPNClassifier** combines all core components of NatPN into a single model.

---

## Components

### **Encoder**
- `self.encoder` maps input x to a latent vector z:

z = f(x)

---

### **Decoder (classifier head)**
- `self.classifier(z)` produces logits for each class.
- Applying softmax gives class proportions:

χ(x)

A vector of non-negative entries that sum to 1.

Interpretation:
- χ(x) represents the **direction** in which evidence is allocated across classes.

---

### **Flow density**
- `self.flow.log_prob(z)` computes the density:

log p(z)

This represents how likely z is under the learned latent distribution.

---

## NatPN logic

### **1. Compute latent representation**
z = f(x)

---

### **2. Compute class direction χ(x)**  
χ(x) = softmax(logits)

This answers:
- *“If we had infinite evidence, how would it be distributed among classes?”*

---

### **3. Compute evidence magnitude n(x) from density**

- High density → large n(x) → high confidence  
- Low density → small n(x) → low confidence  

The density controls **how much** evidence to assign.

---

### **4. Convert to per-class evidence**

evidence(x) = n(x) · χ(x)

Each class receives a portion of the total evidence.

---

### **5. Add a Dirichlet prior α_prior**

This defines the default uncertainty when no evidence is present.

We use a **uniform prior**, i.e. every class starts with the same α value.

---

### **6. Final posterior Dirichlet parameters**

α(x) = α_prior + evidence(x)

These α(x) parameters define the final **Dirichlet posterior** for classification and uncertainty estimation.

---

## Implementation detail

`alpha_prior` is stored as a buffer so it automatically moves with:

model.to(device)


In [ ]:
class NatPNClassifier(nn.Module):
    """Natural Posterior Network for classification with a Dirichlet posterior over class probabilities."""

    def __init__(
        self,
        num_classes: int = 10,
        latent_dim: int = 2,
        flow_length: int = 4,
        certainty_budget: float | None = None,
        n_prior: float | None = None,
    ) -> None:
        """Initialize the NatPN classifier and its components."""
        super().__init__()

        self.num_classes = num_classes
        self.latent_dim = latent_dim

        # 1. Encoder: x -> z
        self.encoder = Encoder2(latent_dim=latent_dim)

        # 2. Decoder: z -> logits for each class (SMOOTHED)
        self.classifier = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
        )

        # 3. Single normalizing flow density over z
        self.flow = RadialFlowDensity(dim=latent_dim, flow_length=flow_length)

        # 4. Certainty budget N_H: scales the density into "evidence"
        #    Intuition: total evidence mass to distribute over the latent space.
        if certainty_budget is None:
            certainty_budget = float(latent_dim)
        self.certainty_budget = certainty_budget

        # 5. Prior pseudo-count n_prior and prior χ_prior
        if n_prior is None:
            n_prior = float(num_classes)

        # χ_prior: uniform over classes
        chi_prior = torch.full((num_classes,), 1.0 / num_classes)  # [C]
        alpha_prior = n_prior * chi_prior  # [C] -> Dirichlet(1,1,...,1)

        # Register as buffer so it is moved automatically with model.to(device)
        self.register_buffer("alpha_prior", alpha_prior)

    def forward(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Forward pass.

        Args:
            x: Input batch, shape [B, 1, 28, 28] for MNIST.

        Returns:
            alpha: Posterior Dirichlet parameters, shape [B, C].
            z: Latent representation, shape [B, latent_dim].
            log_pz: Log-density log p(z) under the flow, shape [B].
        """
        # Encode to latent space
        z = self.encoder(x)  # [B, latent_dim]

        # Class logits -> per-class χ (like normalized preferences)
        logits = self.classifier(z)  # [B, C]
        chi = torch.softmax(logits, dim=-1)  # [B, C], sums to 1

        # Flow density over z -> log p(z)
        log_pz = self.flow.log_prob(z)  # [B]

        # Convert density into scalar evidence n(x)
        # n(x) = N_H * exp(log p(z)) = N_H * p(z)
        n = self.certainty_budget * log_pz.exp()  # [B], evidence ≥ 0
        n = torch.clamp(n, min=1e-8)  # avoid exact zero for numerical stability

        # Evidence per class: n_i * χ_i  -> pseudo-counts
        evidence = n.unsqueeze(-1) * chi  # [B, C]

        # Posterior Dirichlet parameters: alpha = alpha_prior + evidence
        alpha = self.alpha_prior.unsqueeze(0) + evidence  # [B, C]

        return alpha, z, log_pz

# 4. NatPN Loss, Uncertainty & Unified Training Function

We now define:

1. **A NatPN loss function** that matches the Bayesian interpretation.

2. **Helper functions** to convert α into predictions and uncertainty measures.

3. **A unified training function** that hides the training boilerplate, similar to your unified evidential training example.


# 4.1 NatPN Loss for Classification (Dirichlet + Categorical)

## Explanation

For classification, NatPN uses a **Dirichlet posterior** over class probabilities.

---

## Given:

- True label **y ∈ {0, …, C−1}**
- Posterior Dirichlet parameters **α ∈ ℝᶜ₊₀**
- Total concentration **α₀ = Σᶜ αᶜ**

---

## Expected negative log-likelihood under the Dirichlet:

Eₚ∼Dir(α)[−log πᵧ] = ψ(α₀) − ψ(αᵧ)

where:

- **ψ** is the **digamma function** (derivative of log-Gamma).  
- This term encourages α to place more mass on the correct class.

---

## Entropy regularizer to avoid overconfident posteriors:

H(Dir(α)) = entropy of the Dirichlet distribution

---

## Loss per sample:

L = ψ(α₀) − ψ(αᵧ) − λ · H(Dir(α))

(where the first part is the expected NLL)

---

## λ = entropy_weight is a hyperparameter:

- Larger **λ** → push towards **higher entropy** (more conservative uncertainty).  
- Smaller **λ** → allow **more confident** (low-entropy) posteriors.

---

We average this loss over the batch.


In [ ]:
def natpn_loss2(
    alpha: torch.Tensor,
    y: torch.Tensor,
    entropy_weight: float = 1e-4,
) -> torch.Tensor:
    """NatPN classification loss based on a Dirichlet-Categorical Bayesian formulation.

    Args:
        alpha: Posterior Dirichlet parameters, shape [B, C].
        y: Ground-truth class labels, shape [B] with values in [0, C-1].
        entropy_weight: λ controlling the strength of the entropy regularizer.

    Returns:
        Scalar loss tensor.
    """
    # Total concentration alpha0 per sample
    alpha0 = alpha.sum(dim=-1)  # [B]

    # Digamma function
    digamma = torch.digamma

    # Expected negative log-likelihood for each sample:
    # E[-log p(y)] = ψ(alpha0) - ψ(alpha_y)
    idx = torch.arange(y.size(0), device=y.device)
    expected_nll = digamma(alpha0) - digamma(alpha[idx, y])  # [B]

    # Entropy of Dirichlet posterior
    dir_dist = Dirichlet(alpha)
    entropy = dir_dist.entropy()  # [B]

    loss = (expected_nll - entropy_weight * entropy).mean()
    return loss

# 4.2 Predictions & Uncertainty from Dirichlet

## Explanation

Given a Dirichlet posterior **Dir(α)**, we can extract:

---

## 1. Posterior mean class probabilities

p̂₍c₎ = α₍c₎ / α₀

---

## 2. Predicted class

argmax₍c₎ p̂₍c₎

---

## 3. Predictive entropy (of the mean probabilities)

H(p̂) = − Σ₍c₎ p̂₍c₎ log p̂₍c₎

- Low entropy → confident prediction  
- High entropy → uncertain prediction  

---

## 4. Total evidence α₀

- Large α₀ → high overall evidence → **small epistemic uncertainty**  
- Small α₀ → little evidence → **high epistemic uncertainty**

We use a simple epistemic uncertainty proxy:

epi(x) = 1 / (1 + α₀)

- When α₀ is big (confident), epi is small.  
- When α₀ is small (uncertain), epi is large.


In [ ]:
def natpn_predict(alpha: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
    """Compute class predictions and posterior mean probabilities from Dirichlet alpha."""
    eps = 1e-8
    alpha0 = alpha.sum(dim=-1, keepdim=True)  # [B, 1]
    probs = alpha / (alpha0 + eps)  # [B, C]
    preds = probs.argmax(dim=-1)  # [B]
    return preds, probs


def natpn_uncertainty(
    alpha: torch.Tensor,
) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """Custom uncertainty.

    - pred_entropy: base entropy of mean probs
                    + evidence-basiertes Boosting (low alpha0 -> viel höher)
    - epistemic_proxy: einfache 1/(1+alpha0)
    - alpha0: total evidence
    """
    eps = 1e-8

    # Total evidence
    alpha0 = alpha.sum(dim=-1)  # [B]

    # Posterior mean probabilities
    probs = alpha / (alpha0.unsqueeze(-1) + eps)  # [B, C]

    # 1) Basis-Entropie der mean-Probs
    base_entropy = -(probs * torch.log(probs + eps)).sum(dim=-1)  # [B]

    # 2) Evidence-basierter Boost:
    #    ID: alpha0 ~ max -> term ~ 0
    #    OOD: alpha0 << max -> term > 0 (stark)
    max_alpha0 = alpha0.max().clamp(min=1.0)  # Skala
    evidence_term = -torch.log(alpha0 / max_alpha0 + eps)  # [B]

    # Stärke des Boosts (kannst du anpassen, falls du noch mehr Unterschied willst)
    beta = 1.0

    pred_entropy = base_entropy + beta * evidence_term  # [B]

    # 3) einfacher epistemischer Proxy
    epistemic_proxy = 1.0 / (1.0 + alpha0)  # [B]

    return pred_entropy, epistemic_proxy, alpha0

# 4.3 Unified NatPN Training Function (torch.py-style)

## Explanation

This function is the NatPN analogue of your **Unified Evidential Training Function**.

---

## Inputs:

- **model**: a NatPN model that returns **α** when called on **x**  
- **dataloader**: yields **(x, y)** batches  
- **loss_fn**: here **natpn_loss**  
- Training hyperparameters: **epochs, lr, device**

---

## Inside:

- Move model to the correct device  
- Use Adam optimizer (standard choice)

For each epoch:

- Loop over batches  
- Send data to device  
- Compute α = model(x)  
- Compute loss: loss_fn(α, y)  
- Backpropagate and update model parameters  
- Track total loss to print average per epoch  

---

This is exactly the kind of function you may want to wrap later inside your:

probly.train.natpn.torch

module.


In [ ]:
from collections.abc import Callable


def unified_natpn_train(
    model: nn.Module,
    dataloader: DataLoader,
    loss_fn: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    epochs: int = 5,
    lr: float = 1e-3,
    device: str = "cuda",
) -> None:
    """Unified training routine for NatPN models (PyTorch backend).

    Args:
        model: NatPN-like model that returns alpha when called as model(x).
        dataloader: PyTorch DataLoader providing (x, y) batches.
        loss_fn: Loss function taking (alpha, y) -> scalar loss.
        epochs: Number of epochs to train.
        lr: Learning rate for Adam optimizer.
        device: "cuda" or "cpu".
    """
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0

        for batch_x, batch_y in dataloader:
            x = batch_x.to(device)
            y = batch_y.to(device)

            optimizer.zero_grad()

            # Forward: model returns Dirichlet alpha
            alpha, _, _ = model(x)

            # Compute NatPN loss
            loss = loss_fn(alpha, y)

            # Backpropagation
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {avg_loss:.4f}")

# 5. Training Loop, Evaluation & OOD Uncertainty Check

Now we put everything together:

1. **Initialize** the NatPN model with our chosen hyperparameters.  
2. **Train** it on **MNIST**.  
3. Evaluate **accuracy** on the MNIST test set.  
4. Compute and compare **uncertainty stats** for MNIST (ID) and FashionMNIST (OOD).


# 5.1 Initialize Model & Loss, and Train

## Explanation

We:

- Choose simple but reasonable hyperparameters:
  - **latent_dim = 2** for small latent space.  
  - **flow_length = 4** radial flow layers.  
  - **certainty_budget = 2.0** as global scaling factor for evidence.

- Create an instance of **NatPNClassifier**.

- Define **loss_fn** as a thin wrapper around **natpn_loss** with a chosen entropy weight.

- Call **unified_natpn_train** to run several epochs of training.


In [ ]:
import torch

# Choose device again for safety
device = "cuda" if torch.cuda.is_available() else "cpu"

# Hyperparameters
num_classes = 10
latent_dim = 2  # low dimensional latent space
flow_length = 4  # number of radial flow layers
certainty_budget = 2.0  # scales density to evidence

# Create NatPN model
model = NatPNClassifier(
    num_classes=num_classes,
    latent_dim=latent_dim,
    flow_length=flow_length,
    certainty_budget=certainty_budget,
)


# NatPN loss function with chosen entropy weight
def loss_fn(alpha: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    return natpn_loss2(alpha, y, entropy_weight=1e-4)


# Train the model
unified_natpn_train(
    model=model,
    dataloader=train_loader,
    loss_fn=loss_fn,
    epochs=5,
    lr=1e-3,
    device=device,
)

In [ ]:
from probly.losses.evidential.torch import natpn_loss
from probly.models.evidential.torch import NatPNModel
from probly.train.evidential.torch import unified_evidential_train

# hier noch encoder definieren pls shawn

model = NatPNModel(certainty_budget=2.0)
unified_evidential_train(mode="NatPostNet", model=model, dataloader=train_loader, loss_fn=natpn_loss)

# 5.2 Evaluation: Accuracy on MNIST (ID)

## Explanation

To verify that NatPN is not only good at uncertainty but also **predicts well**, we measure:

- **Classification accuracy** on the MNIST test set.

---

We:

- Set the model to **eval()** mode (turns off dropout etc., if used).  
- Loop over **test_loader**.  
- Compute **α** for each batch, then **preds** via **natpn_predict**.  
- Compare predictions to true labels and count correct ones.


In [ ]:
def evaluate_accuracy(
    model: nn.Module,
    dataloader: DataLoader,
    device: str = "cuda",
) -> float:
    """Evaluate classification accuracy of NatPN model.

    Args:
        model: Trained NatPNClassifier.
        dataloader: DataLoader for evaluation.
        device: "cuda" or "cpu".

    Returns:
        Accuracy as float between 0 and 1.
    """
    model.eval()
    model.to(device)

    correct = 0
    total = 0

    with torch.no_grad():
        for batch_x, batch_y in dataloader:
            x = batch_x.to(device)
            y = batch_y.to(device)

            alpha, _, _ = model(x)
            preds, _ = natpn_predict(alpha)

            correct += (preds == y).sum().item()
            total += y.numel()

    acc = correct / total
    print(f"Accuracy on in-distribution test set: {acc:.4f}")
    return acc


# Evaluate on MNIST test set
test_acc = evaluate_accuracy(model, test_loader, device=device)

# 5.3 OOD Check: Are Uncertainty Signals Reasonable?

## Explanation

One of the main claims of NatPN is:

**It should be more uncertain on inputs that are far from the training distribution.**

---

## We test this by:

### 1. Computing uncertainty metrics on:
- **ID:** MNIST test set  
- **OOD:** FashionMNIST test set  

---

### 2. For each dataset, we compute:
- **α** for all samples  
- **pred_entropy:** predictive entropy  
- **α₀:** total evidence  

---

### 3. We compare:
- Mean **α₀** (evidence) for ID vs OOD  
- Mean **pred_entropy** for ID vs OOD  

---

## If NatPN works as intended:

- **ID:** higher evidence, lower entropy  
- **OOD:** lower evidence, higher entropy


In [ ]:
def collect_uncertainty_stats(
    model: nn.Module,
    dataloader: DataLoader,
    device: str = "cuda",
) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """Collect predictive entropy, epistemic proxy and evidence alpha0 across a dataset.

    Args:
        model: NatPN model.
        dataloader: DataLoader with images (labels are ignored).
        device: "cuda" or "cpu".

    Returns:
        pred_entropy_all: Tensor of shape [N], predictive entropy per sample.
        epistemic_all: Tensor of shape [N], epistemic proxy per sample.
        alpha0_all: Tensor of shape [N], total evidence per sample.
    """
    model.eval()
    model.to(device)

    pred_entropy_all = []
    epistemic_all = []
    alpha0_all = []

    with torch.no_grad():
        for batch_x, _ in dataloader:
            x = batch_x.to(device)

            alpha, _, _ = model(x)
            pred_entropy, epistemic_proxy, alpha0 = natpn_uncertainty(alpha)

            pred_entropy_all.append(pred_entropy.cpu())
            epistemic_all.append(epistemic_proxy.cpu())
            alpha0_all.append(alpha0.cpu())

    pred_entropy_all = torch.cat(pred_entropy_all)
    epistemic_all = torch.cat(epistemic_all)
    alpha0_all = torch.cat(alpha0_all)

    return pred_entropy_all, epistemic_all, alpha0_all


# Collect stats on ID (MNIST test) and OOD (FashionMNIST)
id_entropy, id_epi, id_alpha0 = collect_uncertainty_stats(model, test_loader, device=device)
ood_entropy, ood_epi, ood_alpha0 = collect_uncertainty_stats(model, ood_loader, device=device)

print("=== Uncertainty summary ===")
print(f"ID  - mean evidence alpha0: {id_alpha0.mean():.2f}, mean predictive entropy: {id_entropy.mean():.2f}")
print(f"OOD - mean evidence alpha0: {ood_alpha0.mean():.2f}, mean predictive entropy: {ood_entropy.mean():.2f}")

# 5.4 (Optional) Inspect Single-Sample Uncertainty

## Explanation

To make the behavior even more concrete, we inspect:

- One batch of **MNIST (ID)**.  
- One batch of **FashionMNIST (OOD)**.

---

## For each batch we print:

- The first few **labels** (for ID).  
- The first few **α₀** values (evidence).  
- The first few **predictive entropies**.

---

## You should see:

- **ID:** relatively large α₀, smaller entropies.  
- **OOD:** relatively smaller α₀, larger entropies.


In [ ]:
# Example: inspect one batch from ID and one batch from OOD
model.eval()

# One ID batch (MNIST)
x_id, y_id = next(iter(test_loader))
x_id = x_id.to(device)
y_id = y_id.to(device)

with torch.no_grad():
    alpha_id, _, _ = model(x_id)
    pred_entropy_id, epi_id, alpha0_id = natpn_uncertainty(alpha_id)

print("ID batch (MNIST):")
print("  First 10 labels:         ", y_id[:10].tolist())
print("  First 10 evidence alpha0:    ", [float(v.detach()) for v in alpha0_id[:10]])
print("  First 10 pred entropy:   ", [float(v.detach()) for v in pred_entropy_id[:10]])

# One OOD batch (FashionMNIST)
x_ood, y_ood = next(iter(ood_loader))
x_ood = x_ood.to(device)

with torch.no_grad():
    alpha_ood, _, _ = model(x_ood)
    pred_entropy_ood, epi_ood, alpha0_ood = natpn_uncertainty(alpha_ood)

print("\nOOD batch (FashionMNIST):")
print("  First 10 evidence alpha0:    ", [float(v.detach()) for v in alpha0_ood[:10]])
print("  First 10 pred entropy:   ", [float(v.detach()) for v in pred_entropy_ood[:10]])

# 6. Summary

In this notebook-style tutorial, we:

- Built a **Natural Posterior Network (NatPN)** for classification in PyTorch.

- Implemented:
  - A **latent encoder**.  
  - A **radial-flow normalizing flow density** over the latent space.  
  - A **Dirichlet posterior head** that combines prior and evidence.  

- Defined a **Bayesian NatPN loss** that:
  - Minimizes expected negative log-likelihood.  
  - Encourages high-entropy (uncertain) posteriors where appropriate.  

- Computed **uncertainty measures**:
  - Predictive entropy.  
  - Evidence α₀ and an epistemic proxy.  

- Checked that the model:
  - **Does well on MNIST** (accuracy).  
  - Is **more uncertain on FashionMNIST (OOD)** than on MNIST (ID), which demonstrates that the loss and density-based evidence behave reasonably with OOD data.  
